In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

在GCP上的端到端机器学习：MLOps阶段2：实验：开始使用Vertex AI TensorBoard

<table align="left">
   <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_tensorboard.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_tensorboard.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_vertex_tensorboard.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何在Google Cloud上使用Vertex AI进行端到端的生产MLOps。本教程涵盖了第二阶段：实验：开始使用Vertex AI TensorBoard。

### 目标

在本教程中，您将学习如何在使用`Vertex AI`进行训练时使用`Vertex AI TensorBoard`。

本教程使用以下Google Cloud ML服务：

- `Vertex AI TensorBoard`
- `Vertex AI Training`

执行的步骤包括：

- 在训练模型时创建一个TensorBoard回调。
- 使用TensorBoard与本地训练的模型。
- 使用`Vertex AI TensorBoard`与`Vertex AI Training`。

### 推荐

在Google Cloud上进行端到端MLOps时，以下是使用TensorBoard可视化训练的最佳实践。

#### 本地TensorBoard

在本地进行临时训练时，使用TensorBoard的OSS版本，无论是命令行版还是守护进程版。

#### 云端TensorBoard

在云端进行训练时，请使用tensorboard.dev，除非有隐私问题。

#### 实验

当存在隐私问题或需要对不同实验配置的结果进行比较时，请使用Vertex AI TensorBoard。

数据集

在本教程中，您将使用MNIST数据集。该数据集的版本内置在TF.Keras框架中。该数据集预测图像是哪个数字，介于0到9之间。

费用

本教程使用 Google Cloud 的计费组件：

- Vertex AI
- Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 来根据您的预计使用量生成费用估算。

### 配置本地开发环境

**如果您正在使用Colab或Vertex AI Workbench笔记本**，您的环境已经满足运行此笔记本的所有要求。

**否则**，请确保您的环境满足这个笔记本的要求。
您需要以下内容：

* Google Cloud SDK
* Git
* Python 3
* virtualenv
* 在带有Python 3的虚拟环境中运行的Jupyter笔记本

Google Cloud指南[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了一套简要的指导：

1. [安装和初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)

2. [安装Python 3。](https://cloud.google.com/python/setup#installing_python)

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，请在终端shell中运行`pip3 install jupyter`命令。

5. 要启动Jupyter，请在终端shell中运行`jupyter notebook`命令。

6. 在Jupyter Notebook仪表板中打开此笔记本。

## 安装

安装以下软件包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install -U tensorflow==2.8 $USER_FLAG -q
! pip3 install --upgrade google-cloud-aiplatform[tensorboard] $USER_FLAG -q

### 重新启动内核

安装完附加包之后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

在您开始之前

设置您的Google Cloud项目

**无论您使用什么笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建账户时，您将获得一个计算/存储成本为300美元的免费信用额度。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

4. 如果您在本地运行这个笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保
Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter会将以`!`为前缀的行视为shell命令，并将以`$`为前缀的Python变量插入这些命令中。

设定你的项目ID

**如果你不知道你的项目ID**，你可以使用`gcloud`获取你的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，该变量用于笔记本其余部分的操作。以下是Vertex AI支持的区域。我们建议您选择最靠近您的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能不能使用多区域存储桶进行Vertex AI的训练。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您在一个现场教程会话中，您可能会使用一个共享的测试账户或项目。为了避免用户在创建的资源之间发生名称冲突，您为每个实例会话创建一个时间戳，并将该时间戳附加到您在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google Cloud账户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经经过身份验证。请跳过此步骤。

如果您正在使用Colab，请运行下面的单元格，并按照提示进行身份验证以通过oAuth授权您的帐户。

否则，请按照以下步骤操作：

1. 在Cloud Console中，转到[**创建服务帐号密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务帐号**。

3. 在**服务帐号名称**字段中输入名称，然后点击**创建**。

4. 在**将此服务帐号授予项目访问权限**部分，点击**角色**下拉列表。在过滤框中键入“Vertex AI”，然后选择**Vertex AI管理员**。在过滤框中键入“存储对象管理员”，然后选择**存储对象管理员**。

5. 点击*创建*。包含您密钥的JSON文件将下载到您的本地环境。

6. 在下面的单元格中将您的服务帐号密钥路径作为`GOOGLE_APPLICATION_CREDENTIALS`变量输入，并运行单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建云存储桶

**无论您的笔记本环境如何，下面的步骤都是必需的。**

初始化Python的Vertex SDK时，您需要指定一个云存储暂存桶。暂存桶是您的数据集和模型资源相关的所有数据在会话之间保留的地方。

在下面设置您的云存储桶的名称。存储桶的名称必须在所有Google Cloud项目中是全局唯一的，包括您组织之外的项目。

In [ ]:
BUCKET_URI = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶不存在时才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

服务账户

**如果您不知道您的服务账户**，请尝试使用`gcloud`命令执行下面的第二个单元格来获取您的服务账户。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform

#### 导入TensorFlow

将TensorFlow包导入到您的Python环境中。

In [ ]:
import tensorflow as tf

初始化用于Python的Vertex AI SDK，用于您的项目和相应的存储桶。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### 设置硬件加速器

您可以为训练设置硬件加速器。

将变量`TRAIN_GPU / TRAIN_NGPU`设置为使用支持GPU的容器映像以及分配给虚拟机（VM）实例的GPU数量。例如，要使用一个具有4个Nvidia Telsa K80 GPU分配给每个VM的GPU容器映像，您可以指定：

（aip.AcceleratorType.NVIDIA_TESLA_K80，4）

否则，请指定（无，无）以使用运行在CPU上的容器映像。

了解更多信息，请在您的区域[此处](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)查看硬件加速器支持。

In [ ]:
import os

if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

#### 设置预构建的容器

设置用于训练的预构建Docker容器映像。

- 将变量`TF`设置为容器映像的TensorFlow版本。例如，`2-1`表示版本2.1，`1-15`表示版本1.15。以下列表显示了一些可用的预构建映像：

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.1".replace(".", "-")

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    REGION.split("-")[0], TRAIN_VERSION
)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)

### 设置机器类型

接下来，设置用于训练的机器类型。

- 将变量`TRAIN_COMPUTE`设置为配置用于训练的VM的计算资源。
 - `machine type`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存
     - `n1-highcpu`：每个vCPU 0.9 GB内存
 - `vCPUs`：\[2, 4, 8, 16, 32, 64, 96\]的数量

*注意：以下内容不支持用于训练：*

 - `standard`：2个vCPUs
 - `highcpu`：2、4和8个vCPUs

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

## 使用TensorBoard进行训练

TensorBoard提供了实时可视化训练和结果（指标）的手段。

您可以将TensorBoard与本地训练、云训练以及“Vertex AI Training”一起使用，这被称为“Vertex AI TensorBoard”。

### 使用TensorBoard进行本地训练

在本地或云端使用TensorBoard进行TF.Keras训练时，您需要按照以下步骤操作：

1. 创建一个TensorBoard回调。
2. 将回调传递给`fit()`方法，指定一个本地目录来写入TensorBoard日志。
3. 上传日志目录到TensorBoard。

了解更多关于[TensorBoard回调](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard)。

In [ ]:
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Flatten

LOG_DIR = "./logs"

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = (x_train / 255.0).astype(np.float32)
x_test = (x_test / 255.0).astype(np.float32)

model = Sequential(
    [
        Flatten(),
        Dense(512, activation="relu"),
        Dense(512, activation="relu"),
        Dense(10, activation="softmax"),
    ]
)

model.compile(
    loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)


tensorboard = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR)

history = model.fit(x_train, y_train, epochs=4, batch_size=32, callbacks=[tensorboard])

! ls {LOG_DIR}/train/

使用 `tensorboard dev` 命令上传和共享 TensorBoard 日志，您可以上传您的 TensorBoard 日志并与他人共享。上传后，将返回一个 URL，在浏览器中打开 TensorBoard 实例进行可视化。

*注意：* 您的 TensorBoard 实例是公开可见的。

*注意：* 此单元格仅用于演示目的，必须在终端 shell 中运行。在此示例中，在笔记本内运行时，命令会冻结，因为它在等待交互式的是/否输入。您可以使用 Ctrl C 或内核中断来终止命令。

了解更多关于 [什么是 TensorBoard.dev](https://tensorboard.dev/)。

In [ ]:
! tensorboard dev upload --logdir logs \
  --name "Simple experiment with MNIST" \
  --description "Training results" \
  --one_shot

使用Vertex AI TensorBoard上传TensorBoard日志

您可以通过首先创建一个TensorBoard实例，然后使用`tb-gcp-uploader`命令来上传日志。上传完成后，该命令将返回一个URL，可通过浏览器连接到TensorBoard实例。

了解更多关于[TensorBoard概述](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview)。

In [ ]:
TENSORBOARD_DISPLAY_NAME = "example"
tensorboard = aiplatform.Tensorboard.create(display_name=TENSORBOARD_DISPLAY_NAME)
tensorboard_resource_name = tensorboard.gca_resource.name
print("TensorBoard resource name:", tensorboard_resource_name)

使用命令`tb-gcp-uploader`上传日志，并附上以下参数：

--tensorboard_resource_name：TensorBoard实例的资源名称。

--logdir：TensorBoard日志的位置。

--experiment_name：实验名称（训练运行）。

上传后，获取到TensorBoard实例的URL，以可视化训练/结果。

In [ ]:
if 'EXPERIMENT_NAME' not in globals():
    EXPERIMENT_NAME='example-' + TIMESTAMP

output = !tb-gcp-uploader --tensorboard_resource_name={tensorboard_resource_name} \
  --logdir={LOG_DIR} \
  --experiment_name={EXPERIMENT_NAME} --one_shot=True

url = output[1].split(' ')[-1]

#print(url)

from IPython.display import display, HTML
display(HTML("<a href='" + url + "'>click here for TensorBoard instance</a>"))

### 使用Vertex AI TensorBoard进行自定义训练作业训练

要在自定义训练作业训练中使用Vertex AI TensorBoard，您需要进行以下修改：

**Python训练脚本**:

1. 获取环境变量`AIP_TENSORBOARD_LOG_DIR`的值。这是由Vertex AI训练服务设置的。
2. 创建一个TensorBoard回调，并将log_dir参数设置为`AIP_TENSORBOARD_LOG_DIR`的值。

**CustomTrainingJob**:

1. 在`run()`方法中，添加两个额外的参数：

- `tensorboard`：Vertex TensorBoard实例的资源名称。
- `service_account`：具有访问Cloud Storage和Vertex TensorBoard权限的服务帐号。

### 检查培训包

#### 包布局

在开始培训之前，您将看一看如何为自定义培训任务组装Python软件包。解压后，软件包包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是将软件包安装到 Docker 镜像的操作环境中的指令。

文件 `trainer/task.py` 是用于执行自定义培训任务的 Python 脚本。*注意*，当我们在工作人员池规范中引用它时，我们会将目录斜杠替换为点号 (`trainer.task`) 并丢弃文件后缀 (`.py`)。

#### 包装配

在接下来的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow==2.5.0',\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: example image classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py内容

在下一个单元格中，您将编写自定义训练脚本task.py的内容。总结如下：

- 解析命令行参数以获取训练配置和超参数设置。
 - 从命令行（`--model_dir`）获取保存模型文件的目录，如果未指定，则从环境变量`AIP_MODEL_DIR`获取。
 - 获取Vertex AI TensorBoard实例的资源名称，如果未指定，则从环境变量`AIP_TENSORBOARD_LOG_DIR`获取。

- 加载和预处理MNIST数据集。
- 构建和编译DNN模型。
- 创建一个指向TensorBoard实例的训练回调。
- 使用`fit()`方法训练模型并使用TensorBoard实例的回调。

In [ ]:
%%writefile custom/trainer/task.py
# Single Instance Training for MNIST

import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
import numpy as np

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=1e-4, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--batch_size', dest='batch_size',
                    default=20, type=int,
                    help='Batch size.')
parser.add_argument('--tensorboard_log_dir', dest='tensorboard_log_dir',
                    default=os.getenv('AIP_TENSORBOARD_LOG_DIR'), type=str,
                    help='Log directory for TensorBoard instance')
args = parser.parse_args()


def make_datasets():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = (x_train / 255.0).astype(np.float32)
    x_test  = (x_test  / 255.0).astype(np.float32)
    return (x_train, y_train), (x_test, y_test)


(x_train, y_train), _ = make_datasets()


def build_and_compile_model():
    model = Sequential([
        Flatten(),
        Dense(512, activation='relu'),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(loss='sparse_categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(args.lr),
        metrics=['accuracy'])
    return model

model = build_and_compile_model()


def train_model():
    tensorboard = tf.keras.callbacks.TensorBoard(log_dir=args.tensorboard_log_dir)

    history = model.fit(x_train, y_train, epochs=args.epochs, batch_size=args.batch_size, callbacks=[tensorboard])
    return history

history = train_model()
model.save(args.model_dir)

将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成一个压缩的 tar 文件，然后将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_example.tar.gz

### 创建并运行自定义训练任务

要训练一个自定义模型，您需要执行两个步骤：1）创建一个自定义训练任务，2）运行该任务。

#### 创建自定义训练任务

使用`CustomTrainingJob`类创建一个自定义训练任务，并包括以下参数：

- `display_name`：自定义训练任务的可读名称。
- `container_uri`：训练容器镜像。
- `requirements`：训练容器镜像的软件包要求（例如，pandas）。
- `script_path`：训练脚本的相对路径。

In [ ]:
job = aiplatform.CustomTrainingJob(
    display_name="example_" + TIMESTAMP,
    script_path="custom/trainer/task.py",
    container_uri=TRAIN_IMAGE,
    requirements=["gcsfs==0.7.1", "tensorflow-datasets==4.4"],
)

print(job)

### 准备您的命令行参数

现在为您的自定义训练容器定义命令行参数：

- `args`：传递给设置为容器入口点的可执行文件的命令行参数。
  - `--model-dir`：对于我们的演示，我们使用此命令行参数来指定存储模型工件的位置。
      - 直接：将Cloud Storage位置作为命令行参数传递给您的训练脚本（设置变量`DIRECT = True`），
      - 间接：服务将Cloud Storage位置作为环境变量`AIP_MODEL_DIR`传递给您的训练脚本（设置变量`DIRECT = False`）。在这种情况下，您在作业规范中告诉服务模型工件的位置。
  - `"--epochs=" + EPOCHS`：训练的时代数量。
  - `"--steps=" + STEPS`：每个时代的步数。

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_URI, TIMESTAMP)

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]

#### 运行自定义训练作业

接下来，通过调用方法`run`并传入以下参数来运行自定义作业，以启动训练作业：

- `args`：传递给训练脚本的命令行参数。
- `replica_count`：用于训练的计算实例数量（replica_count = 1 表示单节点训练）。
- `machine_type`：计算实例的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作人员复制品的加速器数量。
- `base_output_dir`：写入模型工件的 Google Cloud 存储位置。
- `tensorboard`：Vertex AI TensorBoard 实例的资源名称。
- `service_account`：具有访问 Google Cloud 存储和 Vertex AI TensorBoard 权限的服务帐号。
- `sync`：是否阻塞直到作业完成。

In [ ]:
if TRAIN_GPU:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        tensorboard=tensorboard_resource_name,
        service_account=SERVICE_ACCOUNT,
        sync=True,
    )
else:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        tensorboard=tensorboard_resource_name,
        service_account=SERVICE_ACCOUNT,
        sync=True,
    )

model_path_to_deploy = MODEL_DIR

查看您的TensorBoard实例

要查看与培训作业相关联的Vertex TensorBoard，请导航到Google Cloud控制台的Vertex AI部分的培训页面。单击培训作业以查看培训详细信息页面，然后单击页面顶部的“打开TensorBoard”按钮。

或者，您可以导航到“实验”选项卡并查看所有实验的列表。您的实验名称与培训作业相同。

### 删除TensorBoard实例

接下来，删除TensorBoard实例。

In [ ]:
tensorboard.delete()

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 模型
- 自定义作业
- 云存储存储桶

In [ ]:
# Delete the custom training job
job.delete()

# Set this to true only if you'd like to delete your bucket
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI